# Extremely Minimal, Dirty Bleak Scanner implementation 

In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio

In [3]:
from bleak import BleakClient, BleakScanner
from bleak.backends.device import BLEDevice

In [4]:
from pycycling.sterzo import Sterzo

In [5]:
# class GattServer:
#     def __init__(self):
#         self.address = address


In [6]:
HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"

In [7]:
class BleakModel:
    '''
    This class is a transitions.AsyncModel wrapper around the BleakScanner class.
    It is used to scan for Bluetooth Low Energy devices and return the results within the state machine framework.
    '''
    bt_devices = {} # class variable to store the discovered devices, since we can only have one BleakScanner
    stop_scan_event = asyncio.Event() # class variable to stop the scan
    def __init__(self):
        self.bleak_client: BleakClient = None
        self.ble_device: BLEDevice = None
        self.connection_target = None
        self.stop_streaming_event = asyncio.Event()
        self.sterzo: Sterzo = None

    def set_target_address(self, address):
        if address in BleakModel.bt_devices:
            self.connection_target = address
            return True
        else:
            return ValueError(f"Address {address} not found in discovered devices")

    async def bt_scan(self):        
        # Start the scanner with the detection callback
        def detection_callback(device, advertisement_data):
            BleakModel.bt_devices[device.address] = device
        async with BleakScanner(detection_callback) as scanner:
            await self.stop_scan_event.wait() # continues to scan until stop_scan_event is set

    async def stop_bt_scan(self):
        self.stop_scan_event.set()

    async def connect_to_device(self):
        if len(BleakModel.bt_devices) == 0:
            return False
       # ble_device = BleakModel.bt_devices[self.connection_target][0] # it's best to feed bleak.BLEDevice objects to the BleakClient
        self.ble_device = (BleakModel.bt_devices.pop(self.connection_target))# remove the device from the list to avoid connecting to it multiple times
        self.bleak_client = BleakClient(self.ble_device) # we don't use the async context manager because we want to access the client object from the disconnect function
        try:
            connected = await self.bleak_client.connect()
            if connected:
                print(f"Connected to {self.connection_target}")
                # Perform operations here, for example, read or write characteristics
                # self.sterzo = Sterzo(self.bleak_client)

                
                return True
            else:
                print(f"Failed to connect to {self.connection_target}")
        except Exception as e:
            print(f"An error occurred: {e}")
            return False
        return True
    
    async def disconnect_from_device(self):
        BleakModel.bt_devices[self.connection_target] = self.ble_device # put it back in the list
        await self.bleak_client.disconnect()
        print(f"Disconnected from {self.bleak_client.address}")

    async def stream_from_device(self):
        self.stop_streaming_event.clear()
        # def steering_handler(steering_angle):
        #     print(steering_angle)
        
        # self.sterzo.set_steering_measurement_callback(steering_handler)
        # await self.sterzo.enable_steering_measurement_notifications()

        def handle_hr_measurement(sender, data):
            print(f"Data received from {sender}: {data}")
            flag = data[0]
            heart_rate = data[1]
            print(f"Heart Rate: {heart_rate} beats per minute")
        
        await self.bleak_client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)
        print("Started streaming")

        await self.stop_streaming_event.wait()
        return True

    async def stop_stream_from_device(self):
        self.stop_streaming_event.set()
        try:
            #await self.sterzo.disable_steering_measurement_notifications()
            await self.bleak_client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)
            print("Stopped streaming")
            return True
        except Exception as e:
            print(f"An error occurred while stopping streaming: {e}")
            return False

    

In [8]:

model = BleakModel()

transitions = []

machine = AsyncMachine(model, states=["Init", "Scanning", "Connected", "Streaming"], transitions=transitions, initial='Init')

machine.add_transition(
    trigger="start_scan",
    source="Init",
    dest="Scanning",
    after="bt_scan"
)

machine.add_transition(
    trigger="stop_scan",
    source="Scanning",
    dest="Init",
    after="stop_bt_scan"
)

machine.add_transition(
    trigger="connect",
    source="Init",
    dest="Connected",
    conditions="connect_to_device",
)

machine.add_transition(
    trigger="disconnect",
    source="Connected",
    dest="Init",
    before="disconnect_from_device"
    
)

machine.add_transition(
    trigger="stream",
    source="Connected",
    dest="Streaming",
    after="stream_from_device"
)

machine.add_transition(
    trigger="stop_stream",
    source="Streaming",
    dest="Connected",
    before="stop_stream_from_device"
)

In [9]:
await model.connect() # should Fail, because we're trying to connect to a device without scanning first
# specifically, `is_device_list_empty` will return True, and the `unless` condition will fail

False

In [10]:
# Since we're in a Jupyter notebook, 
# we need to use `create_task()` instead of `get_event_loop().run_until_complete()`
# to use the existing event loop which Jupyter runs on.
asyncio.create_task(model.start_scan())
# if we used
# `await model.start_scan()`
# then it would work, but it would block until the scan is finished, which is not what we want,
# because below we will show how the `model.bt_devices` dictionary gets filled with the results of the scan.

<Task pending name='Task-7' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

In [11]:
async def stop_after_delay(delay, model):
    print("Starting sleep for 2 seconds (giving the scanner time to scan)")
    await asyncio.sleep(delay)
    print("Slept for 2 seconds (sending end signal to scanner)")
    await model.stop_scan()
    print("Stopped scanning")
    

asyncio.create_task(stop_after_delay(2, model))

<Task pending name='Task-9' coro=<stop_after_delay() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_3292/921128545.py:1>>

In [12]:
async def print_as_we_go():
    for i in range(20):
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

Starting sleep for 2 seconds (giving the scanner time to scan)
0 devices found in 0.0 seconds.
2 devices found in 0.05 seconds.
3 devices found in 0.1 seconds.
3 devices found in 0.15 seconds.
3 devices found in 0.2 seconds.
3 devices found in 0.25 seconds.
3 devices found in 0.3 seconds.
3 devices found in 0.35 seconds.
4 devices found in 0.4 seconds.
4 devices found in 0.45 seconds.
4 devices found in 0.5 seconds.
4 devices found in 0.55 seconds.
4 devices found in 0.6 seconds.
5 devices found in 0.65 seconds.
5 devices found in 0.7 seconds.
5 devices found in 0.75 seconds.
5 devices found in 0.8 seconds.
5 devices found in 0.85 seconds.
5 devices found in 0.9 seconds.
8 devices found in 0.95 seconds.


In [13]:
model.bt_devices

{'DC07A1C7-F223-9800-4F6C-2A4BDFE90E15': BLEDevice(DC07A1C7-F223-9800-4F6C-2A4BDFE90E15, None),
 '782494C8-9307-118C-9A93-23059622886A': BLEDevice(782494C8-9307-118C-9A93-23059622886A, None),
 'D064C2BC-81D4-15E2-F56D-197A78B89F89': BLEDevice(D064C2BC-81D4-15E2-F56D-197A78B89F89, None),
 '55E0C124-F531-6DEB-DAFB-8DFF4AFB11BF': BLEDevice(55E0C124-F531-6DEB-DAFB-8DFF4AFB11BF, None),
 '51BC6973-50AA-D8BE-B481-0033AC8B817B': BLEDevice(51BC6973-50AA-D8BE-B481-0033AC8B817B, None),
 'EF759457-7230-5B90-6F36-523344CBD827': BLEDevice(EF759457-7230-5B90-6F36-523344CBD827, None),
 'CE38A319-3910-6E64-516C-E54E05550C30': BLEDevice(CE38A319-3910-6E64-516C-E54E05550C30, None),
 '14863C4D-BF71-4EA3-C6B4-98001056AAF8': BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO)}

In [14]:
model.set_target_address("14863C4D-BF71-4EA3-C6B4-98001056AAF8")

True

In [15]:
await asyncio.sleep(3)

Slept for 2 seconds (sending end signal to scanner)
Stopped scanning


In [16]:
await model.connect()

Connected to 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [17]:
model.state

'Connected'

In [18]:
model.connection_target

'14863C4D-BF71-4EA3-C6B4-98001056AAF8'

In [19]:
# a second bluetooth model

model2 = BleakModel()
machine.add_model(model2)

In [20]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-15' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_3292/3697076873.py:1>>

In [21]:
await asyncio.sleep(10)

Started streaming
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00P')
Heart Rate: 80 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00P')
Heart Rate: 80 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00P')
Heart Rate: 80 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00P')
Heart Rate: 80 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00Q')
Heart Rate: 81 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00Q')
Heart Rate: 81 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'

In [22]:
model.state

'Streaming'

In [23]:
await model.stop_stream()

Stopped streaming


True

In [24]:
await model.disconnect()

Disconnected from 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

# Important Todo

After stream is stopped, we can't simpy re-start it.

We need to re-create a BleakClient object, which means we need to retreat back to Init state
in order to call `connect()`, which re-creates that BleakClient object.

In our FSM, we need to remove the path from Connected -> Streaming because that's not possible.

In [25]:
model.state

'Init'

In [26]:
await model.connect()

Connected to 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [27]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-24' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_3292/3697076873.py:1>>

Started streaming
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00S')
Heart Rate: 83 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00S')
Heart Rate: 83 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00S')
Heart Rate: 83 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10S\x02\x03')
Heart Rate: 83 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10R\xff\x02\xe8\x01\xe8\x01')
Heart Rate: 82 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10Q\xe1\x02')
Heart Rate: 81 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 3